In [ ]:
using Pkg
pkg_folder = "../"
Pkg.activate(pkg_folder)
using BenchmarkTools
import Markdown; Base.showable(::MIME"text/markdown", ::Markdown.MD) = false # to make sth like """?replace"""  works for vscode backend, remember to choose Jupyter kernel
include(pkg_folder*"src/initialize.jl")
Threads.nthreads() # check number of threads

  Activating project at `~/Realizibility_index/bnc_julia`


24

For a trimer binding system, the general case will be include all the possibe states of binding, that is for example, for E, X1, X2 binding systems, we allow the dimer formed by {E,X1}, {E,X2}, and {X1,X2},  which gives C1, C2, C3; while also, we allow the trimer formed by {C1,X2}, {C2,X1}, and {C3,E}, which all gives C4. We define the binding constants as {K1,K2,K3,K41,K42,K43}, so we could write the N as:

In [45]:
  #  E X1 X2 C1 C2 C3 C4
N = [1 1  0  -1 0  0  0;  # E + X1 <-> C1, K1
     1 0  1  0  -1 0  0;   # E + X2 <-> C2 , K2
     0 1  1  0  0 -1  0;    # X1 + X2 <-> C3, K3
    #  0 0  1  1  0  0 -1;   # C1 + X2 <-> C4, K41
    #  0 1  0  0  1  0 -1;    # C2 + X1 <-> C4, K42
     1 0  0  0  0  1 -1]   # C3 + E <-> C4, K43

4×7 Matrix{Int64}:
 1  1  0  -1   0   0   0
 1  0  1   0  -1   0   0
 0  1  1   0   0  -1   0
 1  0  0   0   0   1  -1

So actually N got only rank 4.

In [46]:
rank(N)

4

In [47]:
-0.6802706851771759 *3

-2.0408120555315277

In [48]:
[norm_vec_space(v) for v in eachcol(nullspace(N'))]

Vector{Float64}[]

In [49]:
nullspace(N')'

0×4 adjoint(::Matrix{Float64}) with eltype Float64

The above is equivalent to K1K41 = K2K42 = K3K43. # From Nemo

In [50]:
U,S,Vt = svd(N)
r = sum(S .> 1e-12)   # rank estimate
Q = Vt[1:r,:]

4×4 Matrix{Float64}:
 -0.688191   0.353553      3.13212e-16   0.16246
 -0.425325  -0.353553      0.5          -0.262866
 -0.425325  -0.353553     -0.5          -0.262866
  0.262866   1.25631e-16  -0.5          -0.425325

In [43]:
U,S,Vt = svd(N)
r = sum(S .> 1e-12)   # rank estimate
Q = Vt[1:r,:]

4×6 Matrix{Float64}:
 -0.5          -0.42009       0.271154     -0.353553  -0.523376   -0.292564
 -0.5           0.42009      -0.271154     -0.353553   0.445026   -0.409604
 -0.5           0.271154      0.42009       0.353553   0.150889    0.358495
  5.55112e-16  -5.55112e-17   2.77556e-17   0.707107  -0.0783504  -0.702169

In [51]:
S

4-element Vector{Float64}:
 2.288245611270737
 2.0
 1.4142135623730956
 0.8740320488976422

Actually we can use Nemo lll function to give a pretty bases, but the most efficient basis will be the above orthognal basis from SVD.

For R-index calculation, maybe we can just pick from the above basis, but should we considering the singualr values?
Seems not, as that depends on the specific bases choice.

The question behind this choose is what is the actually basis for K?

How will that affect the R-index calculation?

Constrains of K is actually make the K into some sub-subspace.

Seems under such cases the R-index depends on what species are we allowed.

In [52]:
  #  E X1 X2 C1 C2 C3 C4
  N = [1 1  0  -1 0  0  0;  # E + X1 <-> C1, K1
       1 0  1  0  -1 0  0;   # E + X2 <-> C2 , K2
            0 1  0  0  1 -1  0;    # X1 + C2 <-> C3, K3
                 0 0  1  1  0  1 -1;    # C1 + X2 <-> C4, K43
                      0 0  0  0  0  1 -1]   # C3 <-> C4, K
                      


5

In [ ]:
N = [1 1  0  -1 0  0  0;  # E + X1 <-> C1, K1
     1 0  1  0  -1 0  0;   # E + X2 <-> C2 , K2
     0 1  0  0  1 -1  0;    # X1 + C2 <-> C3, K3
     0 0  1  1  0  1 -1;    # C1 + X2 <-> C4, K43
     0 0  0  0  0  1 -1]   # C3 <-> C4, K

rank(N)

THe below is to test whether the independent binding is equivalent to remove the colomn of some rows

In [53]:
    # E S P C1 C2 C3
N = [ 1 1 0 -1 0  0;
      1 0 1  0 -1 0;
      0 1 0  0 1 -1]
model = Bnc(N=N)

Bnc{Int8}([1 1 … 0 0; 1 0 … -1 0; 0 1 … 1 -1], [1 0 … 1 1; 0 1 … 0 1; 0 0 … 1 1], 3, 6, 3, Num[x₁, x₂, x₃, x₄, x₅, x₆], Num[q₁, q₂, q₃], Num[K₁, K₂, K₃], nothing, Vector{Int8}[], Dict{Vector{Int8}, Int64}(), Bool[], Int8[], nothing, Vertex[], Bool[], false, Dict{Vector{Int8}, Tuple{SparseMatrixCSC{Float64, Int64}, Int8}}(), -1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.6020599913279624, 0.47712125471966244, 0.47712125471966244, 0.0, 0.0, 0.0], false, sparse([1, 2, 3, 1, 2, 1, 3, 1, 2, 3], [1, 2, 3, 4, 4, 5, 5, 6, 6, 6], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 3, 6), sparse([1, 2, 3, 1, 2, 1, 3, 1, 2, 3], [1, 2, 3, 4, 4, 5, 5, 6, 6, 6], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 3, 6), [[1, 4, 5, 6], [2, 4, 6], [3, 5, 6]], [1, 4, 6, 8], sparse([1, 2, 1, 3, 2, 1, 2, 3, 3], [1, 1, 2, 2, 3, 4, 5, 5, 6], [1, 1, 1, 1, 1, -1, -1, 1, -1], 3, 6), sparse([1, 4, 5, 2, 4, 6, 3, 5, 1, 2, 4, 1, 3, 5, 6, 1, 2, 3, 6], [1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 

In [54]:
find_all_vertices!(model)

---------------------Start finding all vertices, it may takes a while.--------------------
Finished, with 25 vertices found and 25 asymptotic vertices.

-------------Start calculating nullity for each vertex, it also takes a while.------------
1.Building Nρ_inv cache in parallel...
2.Calculating nullity for each vertex in parallel...
3.Storing all vertices information...
Done.


25-element Vector{Vector{Int8}}:
 [1, 2, 3]
 [1, 2, 5]
 [1, 2, 6]
 [1, 4, 3]
 [1, 4, 5]
 [1, 4, 6]
 [1, 6, 3]
 [1, 6, 5]
 [1, 6, 6]
 [4, 2, 3]
 [4, 2, 5]
 [4, 2, 6]
 [4, 4, 3]
 [4, 4, 5]
 [4, 4, 6]
 [5, 2, 3]
 [5, 2, 5]
 [5, 4, 3]
 [5, 4, 5]
 [5, 6, 3]
 [5, 6, 5]
 [6, 2, 3]
 [6, 2, 6]
 [6, 6, 3]
 [6, 6, 6]

In [56]:
vtx = get_vertices(model)

25-element Vector{Vector{Int8}}:
 [1, 2, 3]
 [1, 2, 5]
 [1, 2, 6]
 [1, 4, 3]
 [1, 4, 5]
 [1, 4, 6]
 [1, 6, 3]
 [1, 6, 5]
 [1, 6, 6]
 [4, 2, 3]
 [4, 2, 5]
 [4, 2, 6]
 [4, 4, 3]
 [4, 4, 5]
 [4, 4, 6]
 [5, 2, 3]
 [5, 2, 5]
 [5, 4, 3]
 [5, 4, 5]
 [5, 6, 3]
 [5, 6, 5]
 [6, 2, 3]
 [6, 2, 6]
 [6, 6, 3]
 [6, 6, 6]

In [57]:
Hs = [get_H!(model, v) for v in vtx]

----------------Start calculating vertices neighbor graph, It may takes a while.----------------


LoadError: TaskFailedException

[91m    nested task error: [39mBoundsError: attempt to access 24-element Vector{Vector{Tuple{Int64, VertexEdge{Int8}}}} at index [25]
    Stacktrace:
     [1] [0m[1mthrow_boundserror[22m[0m[1m([22m[90mA[39m::[0mVector[90m{Vector{Tuple{Int64, VertexEdge{Int8}}}}[39m, [90mI[39m::[0mTuple[90m{Int64}[39m[0m[1m)[22m
    [90m   @[39m [90mBase[39m [90m./[39m[90m[4messentials.jl:15[24m[39m
     [2] [0m[1mgetindex[22m
    [90m   @[39m [90m./[39m[90m[4messentials.jl:919[24m[39m[90m [inlined][39m
     [3] [0m[1mmacro expansion[22m
    [90m   @[39m [90m~/Realizibility_index/bnc_julia/[39m[90m[4mbnc_regimes.jl:214[24m[39m[90m [inlined][39m
     [4] [0m[1m(::var"#_calc_vertices_graph_from_perms##34#_calc_vertices_graph_from_perms##35"{var"#_calc_vertices_graph_from_perms##36#_calc_vertices_graph_from_perms##37"{Int64, Vector{Vector{Tuple{Int64, VertexEdge{Int8}}}}, Int64, Vector{Vector{Tuple{Int64, Int8}}}, Un